In [1]:
import re
import pandas as pd
from collections import OrderedDict

In [2]:
docs_df = pd.read_csv('../../data/law_analyzer_msib2.csv')
docs_df

,_index,_id,_score,_source.Status,_source.No,_source.PeraturanGoId,_source.Tematik,_source.Blocks,_source.TanggalPenetapan,_source.Bidang,_source.Slug,_source.Source,_source.Nomor,_source.BlocksMinioPath,_source.FileMinioPath,_source.Judul,_source.Tahun,_source.TanggalPengundangan,_source.Bentuk,_source.PeraturanId
0,law_analyzer_msib2,76-pmk.03-2010,1.0,Tidak Berlaku,76,https://peraturan.go.id/files/bn159-2010.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2010-03-31 00:00:00.000,NaN,76/pmk.03/2010,jdih.kemenkeu.go.id,76/PMK.03/2010,law_analyzer_parse/01956C5F-E27F-413E-BC82-A5B...,law_analyzer/01956C5F-E27F-413E-BC82-A5B879E81...,Tata Cara Pengajuan dan Penyelesaian Permintaa...,2010,2010-03-31 00:00:00.000,Peraturan Menteri Keuangan,01956C5F-E27F-413E-BC82-A5B879E81125
1,law_analyzer_msib2,227-pmk.05-2011,1.0,Berlaku,227,https://peraturan.go.id/files/bn856-2011.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2011-12-19 00:00:00.000,Hukum Keuangan Negara,227/pmk.05/2011,jdih.kemenkeu.go.id,227/PMK.05/2011,law_analyzer_parse/0DD677AE-42DD-4AE7-8953-71F...,law_analyzer/0DD677AE-42DD-4AE7-8953-71F519FAB...,Penyusunan dan Pengesahan Daftar Isian Pelaksa...,2011,2011-12-19 00:00:00.000,Peraturan Menteri Keuangan,0DD677AE-42DD-4AE7-8953-71F519FAB13C
2,law_analyzer_msib2,38-pmk.05-2012,1.0,Tidak Berlaku,38,https://peraturan.go.id/files/bn289-2012.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2012-03-12 00:00:00.000,NaN,38/pmk.05/2012,jdih.kemenkeu.go.id,38/PMK.05/2012,law_analyzer_parse/611AD6BB-D0CF-41F0-BFF3-350...,law_analyzer/611AD6BB-D0CF-41F0-BFF3-350BF669A...,Tarif Layanan Badan Layanan Umum Institut Agam...,2012,2012-03-12 00:00:00.000,Peraturan Menteri Keuangan,611AD6BB-D0CF-41F0-BFF3-350BF669ABA1
3,law_analyzer_msib2,149-pmk.03-2011,1.0,Berlaku,149,https://peraturan.go.id/files/bn573-2011.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2011-09-12 00:00:00.000,Hukum Keuangan Negara,149/pmk.03/2011,jdih.kemenkeu.go.id,149/PMK.03/2011,law_analyzer_parse/3413A28E-D9D1-49E8-9AC6-C64...,law_analyzer/3413A28E-D9D1-49E8-9AC6-C64022DAC...,Sensus Pajak Nasional.,2011,2011-09-12 00:00:00.000,Peraturan Menteri Keuangan,3413A28E-D9D1-49E8-9AC6-C64022DACD32
4,law_analyzer_msib2,20-pmk.04-2018,1.0,Tidak Berlaku,20,https://peraturan.go.id/files/bn320-2018.pdf,['Unclassified'],[{'ContentText': [{'AdditionalContext': [{'Tex...,2018-02-26 00:00:00.000,NaN,20/pmk.04/2018,jdih.kemenkeu.go.id,20/PMK.04/2018,law_analyzer_parse/D3826423-5898-4C28-A8D8-267...,law_analyzer/D3826423-5898-4C28-A8D8-2670E7714...,Perubahan atas Peraturan Menteri Keuangan Nomo...,2018,2018-02-26 00:26:00.000,Peraturan Menteri Keuangan,D3826423-5898-4C28-A8D8-2670E77149E8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,law_analyzer_msib2,pp-81-tahun-2012,1.0,Berlaku,81,https://peraturan.go.id/files/pp81-2012bt.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2012-10-12 00:00:00.000,NaN,pp-81-tahun-2012,jdih.kemenkeu.go.id,PP 81 TAHUN 2012,law_analyzer_parse/490A3E49-26EC-4CEB-A0B6-CE3...,law_analyzer/490A3E49-26EC-4CEB-A0B6-CE37BCBD9...,Pengelolaan Sampah Rumah Tangga dan Sampah Sej...,2012,2012-10-12 00:00:00.000,Peraturan Pemerintah,490A3E49-26EC-4CEB-A0B6-CE37BCBD961E
239,law_analyzer_msib2,172-pmk.08-2017,1.0,Tidak Berlaku,172,https://peraturan.go.id/files/bn1679-2017.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalContext': [], 'R...",2017-11-23 00:00:00.000,NaN,172/pmk.08/2017,jdih.kemenkeu.go.id,172/PMK.08/2017,law_analyzer_parse/07EF544E-9C1A-4C34-BE97-3DF...,law_analyzer/07EF544E-9C1A-4C34-BE97-3DF4B2DC1...,Tata Cara Pengadaan Pinjaman Luar Negeri Tunai.,2017,2017-11-23 00:00:00.000,Peraturan Menteri Keuangan,07EF544E-9C1A-4C34-BE97-3DF4B2DC1A12
240,law_analyzer_msib2,90-pmk.05-2009,1.0,Berlaku,90,https://peraturan.go.id/files/bn98-2009.pdf,['Unclassified'],"[{'ContentText': [{'AdditionalConte

In [3]:
# evaluate the _source.Blocks column
docs_df['_source.Blocks'] = docs_df['_source.Blocks'].apply((lambda x: eval(x) if pd.notnull(x) else x))

In [4]:
# create new dataframe with only the relevant fields (_id, _source.Judul, _source.Blocks)
data_df = docs_df[['_id','_source.Blocks', '_source.Nomor', '_source.Judul', '_source.Tahun', '_source.No']]
data_df

,_id,_source.Blocks,_source.Nomor,_source.Judul,_source.Tahun,_source.No
0,76-pmk.03-2010,"[{'ContentText': [{'AdditionalContext': [], 'R...",76/PMK.03/2010,Tata Cara Pengajuan dan Penyelesaian Permintaa...,2010,76
1,227-pmk.05-2011,"[{'ContentText': [{'AdditionalContext': [], 'R...",227/PMK.05/2011,Penyusunan dan Pengesahan Daftar Isian Pelaksa...,2011,227
2,38-pmk.05-2012,"[{'ContentText': [{'AdditionalContext': [], 'R...",38/PMK.05/2012,Tarif Layanan Badan Layanan Umum Institut Agam...,2012,38
3,149-pmk.03-2011,"[{'ContentText': [{'AdditionalContext': [], 'R...",149/PMK.03/2011,Sensus Pajak Nasional.,2011,149
4,20-pmk.04-2018,[{'ContentText': [{'AdditionalContext': [{'Tex...,20/PMK.04/2018,Perubahan atas Peraturan Menteri Keuangan Nomo...,2018,20
...,...,...,...,...,...,...
238,pp-81-tahun-2012,"[{'ContentText': [{'AdditionalContext': [], 'R...",PP 81 TAHUN 2012,Pengelolaan Sampah Rumah Tangga dan Sampah Sej...,2012,81
239,172-pmk.08-2017,"[{'ContentText': [{'AdditionalContext': [], 'R...",172/PMK.08/2017,Tata Cara Pengadaan Pinjaman Luar Negeri Tunai.,2017,172
240,90-pmk.05-2009,"[{'ContentText': [{'AdditionalContext': [], 'R...",90/PMK.05/2009,Pengelolaan Uang Negara di Bank Indonesia.,2009,90
241,pp-4-tahun-2011,"[{'ContentText': [{'AdditionalContext': [], 'R...",PP 4 TAHUN 2011,Perubahan Struktur Kepemilikan Saham Negara Me...,2011,4


In [5]:
def DetectDefinition(Text):
    Keywords = ["adalah", "merupakan", "yang selanjutnya disebut", "yang selanjutnya disingkat",] #"yang hasil perhitungan"]
    return any(Keyword in Text for Keyword in Keywords)

def ExtractDefinition(Text):
    Parts = Text.split("adalah")
    if len(Parts) == 1:
        Parts = Text.split("merupakan")
    if len(Parts) == 1:
        Parts = Text.split("yang selanjutnya disebut")
    if len(Parts) == 1:
        Parts = Text.split("yang selanjutnya disingkat")
    return Parts[-1].strip()

def DetectDefinitionsInText(Text):
    Sentences = re.split(r'\b\d+\.\s*', Text)
    Sentences = [Sentence.strip()
                for Sentence in Sentences if Sentence.strip()]

    Results = []
    for Sentence in Sentences:
        Sentence = re.sub(r'\s+', ' ', Sentence)

        if DetectDefinition(Sentence):
            Term = None
            if "yang selanjutnya disebut" in Sentence:
                Term = Sentence.split("yang selanjutnya disebut")[0].strip()
            elif "yang selanjutnya disingkat" in Sentence:
                Term = Sentence.split("yang selanjutnya disingkat")[0].strip()
            #elif "yang hasil perhitungan" in Sentence:
            #    Term = Sentence.split("yang hasil perhitungan")[0].strip()
            elif "adalah" in Sentence:
                Term = Sentence.split("adalah")[0].strip()
            elif "merupakan" in Sentence:
                Term = Sentence.split("merupakan")[0].strip()

            Definition = ExtractDefinition(Sentence)

            ShortTerm = None
            if "yang selanjutnya disebut" in Sentence:
                ShortTerm = re.search(
                    r'yang selanjutnya disebut(.*?)adalah', Sentence)
            elif "yang selanjutnya disingkat" in Sentence:
                ShortTerm = re.search(
                    r'yang selanjutnya disingkat(.*?)merupakan', Sentence)

            if ShortTerm:
                ShortTerm = ShortTerm.group(1).strip()

            result = {
                "Text": Sentence,
                "Term": Term,
                "ShortTerm": ShortTerm,
                "Definition": Definition,
                "Type": "DEFINITION"
            }
            Results.append(result)

    return Results

In [6]:
def get_heading(source_blocks):
    chapter_name = []
    part_name = []
    paragraph_name = []
    
    for i in source_blocks:
        # skip if the value is None
        if 'BabContext' in i and i['BabContext'] is not None:
            chapter_name.append(i['BabContext'])
        if 'BagianContext' in i and i['BagianContext'] is not None:
            part_name.append(i['BagianContext'])
        if 'ParagrafContext' in i and i['ParagrafContext'] is not None:
            paragraph_name.append(i['ParagrafContext'])
    
    # combine the list and eliminate duplicate values
    heading_list = chapter_name + part_name + paragraph_name
    heading_list = list(OrderedDict((x, None) for x in heading_list))
    
    return heading_list

def get_definition_term(source_blocks):
    # get first article
    pasal_1 = [content for content in source_blocks if 'Pasal' in content and content['Pasal'] == 'Pasal 1']
    if not pasal_1:
        return [], []
    else:
        pasal_1_content = pasal_1[0]['ContentText']
    
    pasal_1_content_value =  []
    def_term_list = []
    def_short_term_list = []
    
    for val in pasal_1_content:
        if 'Value' in val and val['Value'] is not None:
            pasal_1_content_value.append(val['Value'])
            
    for content in pasal_1_content_value:
        definition_term = DetectDefinitionsInText(content)
        if definition_term:
            term_list = [term['Term'] for term in definition_term]
            short_term_list = [term['ShortTerm'] for term in definition_term]
            
            # eliminate None values from the list
            term_list = list(filter(None, term_list))
            short_term_list = list(filter(None, short_term_list))
            
            def_term_list.extend(term_list)
            def_short_term_list.extend(short_term_list)
    
    return def_term_list, def_short_term_list

def get_keywords(documents_df):
    heading_list = []
    def_term_list = []
    def_short_term_list = []
    
    for idx, row in documents_df.iterrows():
        source_blocks = row['_source.Blocks']
        
        heading = get_heading(source_blocks)
        def_term, def_short_term = get_definition_term(source_blocks)
        
        heading_list.append(heading)
        def_term_list.append(def_term)
        def_short_term_list.append(def_short_term)
    
    # create new column for definition and heading
    documents_df.loc[:, 'definisi'] = def_term_list
    documents_df.loc[:, 'singkatan'] = def_short_term_list
    documents_df.loc[:, 'heading'] = heading_list
    
    return documents_df

In [7]:
data_df = get_keywords(data_df)
data_df

C:\Users\zoxrl\AppData\Local\Temp\ipykernel_2368\3824488815.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_df.loc[:, 'definisi'] = def_term_list
C:\Users\zoxrl\AppData\Local\Temp\ipykernel_2368\3824488815.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_df.loc[:, 'singkatan'] = def_short_term_list
C:\Users\zoxrl\AppData\Local\Temp\ipykernel_2368\3824488815.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

,_id,_source.Blocks,_source.Nomor,_source.Judul,_source.Tahun,_source.No,definisi,singkatan,heading
0,76-pmk.03-2010,"[{'ContentText': [{'AdditionalContext': [], 'R...",76/PMK.03/2010,Tata Cara Pengajuan dan Penyelesaian Permintaa...,2010,76,"[Orang Pribadi Pemegang Paspor Luar Negeri,, B...","[Orang Pribadi,, KPP, KPPN, SPM UP, SKPKPP, SP...","[KETENTUAN UMUM, PERMINTAAN PENGEMBALIAN PAJAK..."
1,227-pmk.05-2011,"[{'ContentText': [{'AdditionalContext': [], 'R...",227/PMK.05/2011,Penyusunan dan Pengesahan Daftar Isian Pelaksa...,2011,227,"[Kuasa Pengguna Anggaran Penerusan Pinjaman, P...",[DIPA-L PP],"[KETENTUAN UMUM, SISA ANGGARAN YANG DILANJUTKA..."
2,38-pmk.05-2012,"[{'ContentText': [{'AdditionalContext': [], 'R...",38/PMK.05/2012,Tarif Layanan Badan Layanan Umum Institut Agam...,2012,38,[Tarif Layanan Badan Layanan Umum Institut Aga...,[],[]
3,149-pmk.03-2011,"[{'ContentText': [{'AdditionalContext': [], 'R...",149/PMK.03/2011,Sensus Pajak Nasional.,2011,149,[Sensus pajak nasional sebagaimana dimaksud pa...,[],[]
4,20-pmk.04-2018,[{'ContentText': [{'AdditionalContext': [{'Tex...,20/PMK.04/2018,Perubahan atas Peraturan Menteri Keuangan Nomo...,2018,20,[],[],[]
...,...,...,...,...,...,...,...,...,...
238,pp-81-tahun-2012,"[{'ContentText': [{'AdditionalContext': [], 'R...",PP 81 TAHUN 2012,Pengelolaan Sampah Rumah Tangga dan Sampah Sej...,2012,81,"[Sampah rumah tangga, Sampah sejenis sampah ru...","[TPS 3R, Pemerintah]","[KETENTUAN UMUM, KEBIJAKAN DAN STRATEGI PENGEL..."
239,172-pmk.08-2017,"[{'ContentText': [{'AdditionalContext': [], 'R...",172/PMK.08/2017,Tata Cara Pengadaan Pinjaman Luar Negeri Tunai.,2017,172,"[Pinjaman Tunai, Pinjaman Program, Pinjaman Si...","[Menteri, Direktur Jenderal]","[KETENTUAN UMUM, RUANG LINGKUP, SUMBER PINJAMA..."
240,90-pmk.05-2009,"[{'ContentText': [{'AdditionalContext': [], 'R...",90/PMK.05/2009,Pengelolaan Uang Negara di Bank Indonesia.,2009,90,"[Bendahara Umum Negara,, Menteri Keuangan, Kua...","[Rekening KUN,]","[KETENTUAN UMUM, SALDO KAS MINIMAL, OPERASIONA..."
241,pp-4-tahun-2011,"[{'ContentText': [{'AdditionalContext': [], 'R...",PP 4 TAHUN 2011,Perubahan Struktur Kepemilikan Saham Negara Me...,2011,4,[],[],[]


In [12]:
data_df.iloc[158]

_id                                                pp-37-tahun-2009
_source.Blocks    [{'ContentText': [{'AdditionalContext': [], 'R...
_source.Nomor                                      PP 37 TAHUN 2009
_source.Judul                                                Dosen.
_source.Tahun                                                  2009
_source.No                                                       37
definisi          [ditjen Peraturan Perundang-undangan1. Dosen, ...
singkatan                                                        []
heading           [KETENTUAN UMUM, SERTIFIKASI, HAK, WAJIB KERJA...
Name: 158, dtype: object

In [13]:
test_blocks = data_df['_source.Blocks'][158]
test_blocks

[{'ContentText': [{'AdditionalContext': [],
    'Ref': None,
    'Type': 'alinea',
    'Value': 'ditjen Peraturan Perundang-undangan1. Dosen adalah pendidik profesional dan ilmuwan dengan tugas utama mentransformasikan, mengembangkan, dan menyebarluaskan ilmu pengetahuan, teknologi, dan seni melalui pendidikan, penelitian, dan pengabdian kepada masyarakat. 2. Dosen Tetap adalah dosen yang bekerja penuh waktu yang berstatus sebagai tenaga pendidik tetap pada satuan pendidikan tinggi tertentu. 3. Satuan Pendidikan Tinggi adalah kelompok layanan pendidikan yang menyelenggarakan pendidikan pada jenjang pendidikan tinggi. 4. Sertifikasi adalah proses pemberian sertifikat pendidik untuk dosen. 5. Sertifikat Pendidik adalah bukti formal sebagai pengakuan yang diberikan kepada dosen sebagai tenaga profesional. 6. Gaji adalah hak yang diterima oleh dosen atas pekerjaannya dari penyelenggara pendidikan tinggi atau Satuan Pendidikan Tinggi dalam bentuk finansial secara berkala sesuai dengan perat

In [14]:
# export test_blocks to json
import json

with open('../../data/test_blocks.json', 'w') as f:
    json.dump(test_blocks, f)

In [9]:
# rename the columns
data_df = data_df.rename(columns={'_id':'id', '_source.Nomor': 'nomor_peraturan', '_source.Judul': 'judul', '_source.Tahun': 'tahun', '_source.No': 'no'})

# convert float to int for tahun and no column
data_df['tahun'] = data_df['tahun'].astype(int)
data_df['no'] = data_df['no'].astype(int)

# drop _source.Blocks column
data_df = data_df.drop(columns=['_source.Blocks'])

data_df

,id,nomor_peraturan,judul,tahun,no,definisi,singkatan,heading
0,76-pmk.03-2010,76/PMK.03/2010,Tata Cara Pengajuan dan Penyelesaian Permintaa...,2010,76,"[Orang Pribadi Pemegang Paspor Luar Negeri,, B...","[Orang Pribadi,, KPP, KPPN, SPM UP, SKPKPP, SP...","[KETENTUAN UMUM, PERMINTAAN PENGEMBALIAN PAJAK..."
1,227-pmk.05-2011,227/PMK.05/2011,Penyusunan dan Pengesahan Daftar Isian Pelaksa...,2011,227,"[Kuasa Pengguna Anggaran Penerusan Pinjaman, P...",[DIPA-L PP],"[KETENTUAN UMUM, SISA ANGGARAN YANG DILANJUTKA..."
2,38-pmk.05-2012,38/PMK.05/2012,Tarif Layanan Badan Layanan Umum Institut Agam...,2012,38,[Tarif Layanan Badan Layanan Umum Institut Aga...,[],[]
3,149-pmk.03-2011,149/PMK.03/2011,Sensus Pajak Nasional.,2011,149,[Sensus pajak nasional sebagaimana dimaksud pa...,[],[]
4,20-pmk.04-2018,20/PMK.04/2018,Perubahan atas Peraturan Menteri Keuangan Nomo...,2018,20,[],[],[]
...,...,...,...,...,...,...,...,...
238,pp-81-tahun-2012,PP 81 TAHUN 2012,Pengelolaan Sampah Rumah Tangga dan Sampah Sej...,2012,81,"[Sampah rumah tangga, Sampah sejenis sampah ru...","[TPS 3R, Pemerintah]","[KETENTUAN UMUM, KEBIJAKAN DAN STRATEGI PENGEL..."
239,172-pmk.08-2017,172/PMK.08/2017,Tata Cara Pengadaan Pinjaman Luar Negeri Tunai.,2017,172,"[Pinjaman Tunai, Pinjaman Program, Pinjaman Si...","[Menteri, Direktur Jenderal]","[KETENTUAN UMUM, RUANG LINGKUP, SUMBER PINJAMA..."
240,90-pmk.05-2009,90/PMK.05/2009,Pengelolaan Uang Negara di Bank Indonesia.,2009,90,"[Bendahara Umum Negara,, Menteri Keuangan, Kua...","[Rekening KUN,]","[KETENTUAN UMUM, SALDO KAS MINIMAL, OPERASIONA..."
241,pp-4-tahun-2011,PP 4 TAHUN 2011,Perubahan Struktur Kepemilikan Saham Negara Me...,2011,4,[],[],[]


In [10]:
# save the dataframe to csv
data_df.to_csv('../../data/law_analyzer_msib2_keywords.csv', index=False)